# 15 P2 Project

* ist1102903 Ana Alfaiate (33%)
  
* ist1102902 Maria Inês Trigueiro (33%)
  
* ist1102881 Raquel Coelho (33%)

Prof. Flávio Martins (and Prof. João Marques)

Lab Shift number: PB03

## PART I – Original Database Schema (SQLite)

##### E-R Diagram

![E-R Diagram](soccer.drawio.png "E-R Diagram")

#### 1. The Original Database Schema

Study the E-R Diagram above corresponding to the original database schema.


Lets download the SQLite database backup/dump!

In [1]:
!wget -c https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download= -O /home/jovyan/data/database.sqlite

--2025-01-09 15:25:00--  https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download=
Resolving github.com (github.com)... failed: Name or service not known.
wget: unable to resolve host address ‘github.com’


#### 2. SQL Queries

Consider the E-R Diagram above and write SQL queries for each of the following information needs.

In [2]:
%reload_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql sqlite:////home/jovyan/data/database.sqlite --alias sqlite

1. What is the maximum number of goals recorded in a match? Output a single number.

In [4]:
%%sql sqlite
SELECT MAX(home_team_goal + away_team_goal) as max_nr_goals_match
FROM Match;

max_nr_goals_match
12


2. List each league name and its number of teams sorted from high to low.

In [15]:
%%sql sqlite

SELECT l.name, count(distinct team_api_id) as nr_teams
FROM league AS l , match, team t
where l.id = league_id and (t.team_api_id = home_team_api_id OR t.team_api_id = away_team_api_id)
group by l.name
order by nr_teams desc

name,nr_teams
France Ligue 1,35
England Premier League,34
Spain LIGA BBVA,33
Italy Serie A,32
Germany 1. Bundesliga,30
Portugal Liga ZON Sagres,29
Netherlands Eredivisie,25
Belgium Jupiler League,24
Poland Ekstraklasa,22
Scotland Premier League,17


In [21]:
%%sql sqlite 
    
SELECT L.name, COUNT(M.team_api_id) AS nr_teams
FROM League AS L JOIN (
    SELECT league_id, home_team_api_id AS team_api_id FROM Match
        UNION 
    SELECT league_id, away_team_api_id AS team_api_id FROM Match) AS M
    ON L.id = M.league_id
GROUP BY L.name
order by nr_teams desc;

name,nr_teams
France Ligue 1,35
England Premier League,34
Spain LIGA BBVA,33
Italy Serie A,32
Germany 1. Bundesliga,30
Portugal Liga ZON Sagres,29
Netherlands Eredivisie,25
Belgium Jupiler League,24
Poland Ekstraklasa,22
Scotland Premier League,17


3. Who are the GoalKeepers (GK) that have played all the away games for their team? Output the `team_long_name` and `player_name` (assuming `match.away_player_1` is the goalkeeper column).

In [29]:
%%sql sqlite
select T.team_long_name, P.player_name
FROM 
    (SELECT * FROM ( SELECT away_team_api_id, away_player_1
                        FROM Match
                        GROUP BY away_team_api_id, away_player_1) 
        GROUP BY away_team_api_id
        HAVING COUNT(*) = 1) s 
    +
    JOIN Player P ON s.away_player_1 = player_api_id 
    join team t on s.away_team_api_id = T.team_api_id;

-- unique ver! versao nao oracle VER OUTRA

team_long_name,player_name
Fortuna Düsseldorf,Fabian Giefer
SV Darmstadt 98,Christian Mathenia
FC Energie Cottbus,Gerhard Tremmel
SC Paderborn 07,Lukas Kruse
RC Recreativo,Asier Riesgo
Watford,Heurelho Gomes
Dijon FCO,Baptiste Reynet
CD Tenerife,Sergio Aragones
DSC Arminia Bielefeld,Dennis Eilhoff


## PART II - New Database Schema (PostgreSQL)

#### 0. Migrate the SQLite Database to PostgreSQL

Instead of writing tedious migration code ourselves, we will be using the `pgloader` CLI tool to get started quickly. The migration task is setup using a `command-file`. Lets see inside that file.

In [3]:
!cat ./command-file.txt

load database
    from '/home/jovyan/data/database.sqlite'
    into postgresql://db:db@postgres/db

with include drop, create tables, create indexes, reset sequences,

prefetch rows = 100, max parallel create index = 1

set work_mem to '16MB', maintenance_work_mem to '512 MB';


The `pgloader` tool will take care of everything automatically including the foreign key constraints (which SQLite does not enforce by default). Lets run it.

In [4]:
!pgloader ./command-file.txt

2025-01-09T15:25:12.010000Z LOG pgloader version "3.6.10~devel"
2025-01-09T15:25:12.010000Z LOG Data errors in '/tmp/pgloader/'
2025-01-09T15:25:12.010000Z LOG Parsing commands from file #P"/home/jovyan/work/command-file.txt"
2025-01-09T15:25:12.120004Z LOG Migrating from #<SQLITE-CONNECTION sqlite:///home/jovyan/data/database.sqlite {1006B26643}>
2025-01-09T15:25:12.120004Z LOG Migrating into #<PGSQL-CONNECTION pgsql://db@postgres:5432/db {1006B267C3}>
2025-01-09T15:25:39.062169Z LOG report summary reset
             table name     errors       rows      bytes      total time
-----------------------  ---------  ---------  ---------  --------------
                  fetch          0          0                     0.000s
        fetch meta data          0         50                     0.090s
         Create Schemas          0          0                     0.000s
       Create SQL Types          0          0                     0.010s
          Create tables          0         14      

Lets take a closer look into the `match` table.

In [7]:
%load_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql postgresql+psycopg://db:db@postgres/db --alias psql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%reload_ext sql

In [10]:
%sqlcmd columns -t match

name,type,nullable,default,autoincrement,comment
id,BIGINT,False,nextval('match_id_seq'::regclass),True,None
country_id,BIGINT,True,None,False,None
league_id,BIGINT,True,None,False,None
season,TEXT,True,None,False,None
stage,BIGINT,True,None,False,None
date,TEXT,True,None,False,None
match_api_id,BIGINT,True,None,False,None
home_team_api_id,BIGINT,True,None,False,None
away_team_api_id,BIGINT,True,None,False,None
home_team_goal,BIGINT,True,None,False,None


Notice how `pgloader` even assigned the safest most-compatible data types for each column so that no data is lost in the migration.

##### 0.1. Introducing a limitation of the original database schema

The `match` table still does not look good to us... Can you try to write a SQL query for the following information need?

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [8]:
%%sql psql

SELECT DISTINCT t.team_long_name
FROM match m
JOIN player p ON (
    p.player_api_id IN (
        m.home_player_1, m.home_player_2, m.home_player_3, m.home_player_4,
        m.home_player_5, m.home_player_6, m.home_player_7, m.home_player_8,
        m.home_player_9, m.home_player_10, m.home_player_11,
        m.away_player_1, m.away_player_2, m.away_player_3, m.away_player_4,
        m.away_player_5, m.away_player_6, m.away_player_7, m.away_player_8,
        m.away_player_9, m.away_player_10, m.away_player_11
    )
)
JOIN team t ON (
    (t.team_api_id = m.home_team_api_id AND p.player_api_id IN (
        m.home_player_1, m.home_player_2, m.home_player_3, m.home_player_4,
        m.home_player_5, m.home_player_6, m.home_player_7, m.home_player_8,
        m.home_player_9, m.home_player_10, m.home_player_11
    )) OR
    (t.team_api_id = m.away_team_api_id AND p.player_api_id IN (
        m.away_player_1, m.away_player_2, m.away_player_3, m.away_player_4,
        m.away_player_5, m.away_player_6, m.away_player_7, m.away_player_8,
        m.away_player_9, m.away_player_10, m.away_player_11
    ))
)
WHERE p.player_name = 'Cristiano Ronaldo';

-- WILL NOT BE EVALUATED - We will be implementing a solution for this problem in the next section.

team_long_name
Manchester United
Real Madrid CF


Closing sqlite

Closing psql

#### 1. Functions, Stored Procedures, and Triggers

Provide the SQL instructions corresponding to each of the following tasks:

1. Create a new `match_player` table to store the data for the starting players (11) for each match (currently stored in the columns `match.home_player_DD` and `match.away_player_DD`). We are going to copy the data contained in the table `match` to `match_player` but we want to use a new column for the player position `position`.

In [9]:
%%sql psql
DROP TABLE IF EXISTS match_player;

CREATE TABLE
  match_player (
    match_id BIGINT,
    team_id BIGINT,
    player_id BIGINT, --- can be null
    position INT,
    type_team ENUM('Away', 'Home'), --     type_team ENUM('Away', 'Home') adicionar OU fazer join com match
    PRIMARY KEY (match_id, team_id, position), -- match_id, player_id (ideal) [not null ]
    FOREIGN KEY (match_id) REFERENCES match(id),
    FOREIGN KEY (team_id) REFERENCES team(team_api_id),
    FOREIGN KEY (player_id) REFERENCES player(player_api_id)
);

++
||
++
++

2. Populate the new table `match_player` using the data contained in the table `match` using a function or stored procedure.

In [42]:
%%sql psql

SELECT * FROM match_player

match_id,team_id,player_id,position
145,9996,39890,1
145,9996,38788,3
145,9996,38312,4
145,9996,26235,5
145,9996,26916,9
145,9996,94289,11
145,8635,34480,1
145,8635,38388,2
145,8635,26458,3
145,8635,13423,4


In [10]:
%%sql psql
DELETE FROM match_player;

CREATE OR REPLACE PROCEDURE populate_match_player()
LANGUAGE plpgsql
AS $$
DECLARE
    match_row match%ROWTYPE;
    position INT;
    player_id INT;
BEGIN
    FOR match_row IN SELECT * FROM match LOOP
        -- HOME players
        FOR position IN 1..11 LOOP
            player_id := CASE position ---confirmar
                WHEN 1 THEN match_row.home_player_1
                WHEN 2 THEN match_row.home_player_2
                WHEN 3 THEN match_row.home_player_3
                WHEN 4 THEN match_row.home_player_4
                WHEN 5 THEN match_row.home_player_5
                WHEN 6 THEN match_row.home_player_6
                WHEN 7 THEN match_row.home_player_7
                WHEN 8 THEN match_row.home_player_8
                WHEN 9 THEN match_row.home_player_9
                WHEN 10 THEN match_row.home_player_10
                WHEN 11 THEN match_row.home_player_11
            END;

            IF player_id IS NOT NULL THEN
                    EXECUTE format(
                        'INSERT INTO match_player (match_id, team_id, player_id, position)
                         VALUES (%s, %s, %s, %s)',
                        match_row.id,
                        match_row.home_team_api_id,
                        player_id,
                        position
                );
            END IF;
        END LOOP;

        -- Insert away players
        FOR position IN 1..11 LOOP
            player_id := CASE position
                WHEN 1 THEN match_row.away_player_1
                WHEN 2 THEN match_row.away_player_2
                WHEN 3 THEN match_row.away_player_3
                WHEN 4 THEN match_row.away_player_4
                WHEN 5 THEN match_row.away_player_5
                WHEN 6 THEN match_row.away_player_6
                WHEN 7 THEN match_row.away_player_7
                WHEN 8 THEN match_row.away_player_8
                WHEN 9 THEN match_row.away_player_9
                WHEN 10 THEN match_row.away_player_10
                WHEN 11 THEN match_row.away_player_11
            END;

            IF player_id IS NOT NULL THEN             

                EXECUTE format(
                    'INSERT INTO match_player (match_id, team_id, player_id, position)
                     VALUES (%s, %s, %s, %s)
                     ', 
                    match_row.id,
                    match_row.away_team_api_id,
                    player_id,
                    position
                );

            END IF;
        END LOOP;
    END LOOP;
END;
$$;

CALL populate_match_player(); -- player_id empty??

++
||
++
++

#### 2. SQL Queries

Consider the new `match_player` table above and write SQL queries for each of the following information needs.

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [11]:
%%sql psql

SELECT DISTINCT(team_id), team_long_name
FROM match_player mp, player p, team t
WHERE mp.player_id = p.player_api_id and p.player_name = 'Cristiano Ronaldo' and t.team_api_id = mp.team_id;

team_id,team_long_name
8633,Real Madrid CF
10260,Manchester United


In [47]:
%%sql psql

SELECT DISTINCT T.team_long_name
FROM Team T JOIN match_player MP ON T.team_api_id = MP.team_id JOIN Player P ON MP.player_id = P.player_api_id
WHERE P.player_name = 'Cristiano Ronaldo';

team_long_name
Manchester United
Real Madrid CF


2. Who are the players that have played all the away games for their team? Output the `team_long_name` and `player_name`.

In [12]:
%%sql psql

SELECT gp.player_id, player_name, t.team_long_name -- division with count 1 s | except 1 min MELHOR MAIS CORRETA
from (
    SELECT away_team_api_id, player_id, count(player_id) as games_player
    from match, match_player mp
    where mp.team_id = away_team_api_id
    group by away_team_api_id, player_id
    order by away_team_api_id) gp 

    JOIN (
    SELECT m.away_team_api_id, COUNT(*) AS total_games_team  
    FROM match m
    GROUP BY 
        m.away_team_api_id
) gt ON gp.away_team_api_id = gt.away_team_api_id, player p , team t
where gp.games_player = gt.total_games_team and player_id = p.player_api_id and gp.away_team_api_id = t.team_api_id

player_id,player_name,team_long_name
13390,Wojciech Skaba,Ruch Chorzów
105526,Adrian Mrowiec,Ruch Chorzów
459662,Volodymyr Tanchyk,Ruch Chorzów
555650,Lukasz Moneta,Ruch Chorzów
166660,Pieter Nys,Oud-Heverlee Leuven
641168,Din Sula,Oud-Heverlee Leuven
138155,Szymon Matuszek,Jagiellonia Białystok
206830,Bartlomiej Pawlowski,Jagiellonia Białystok
412056,Seweryn Michalski,Jagiellonia Białystok
429899,Tomasz Kowalski,Jagiellonia Białystok


In [18]:
%%sql psql
-- podemos colocar mais colunas na tabela match_player?
    
SELECT gp.player_id, player_name, team_long_name --player.player_name
from (
    SELECT away_team_api_id, player_id, count(away_team_api_id) as games_player
    from match, match_player mp
    where mp.team_id = away_team_api_id
    group by away_team_api_id, player_id
    order by away_team_api_id) gp
    JOIN (
    SELECT 
        m.away_team_api_id,
        COUNT(*) AS total_games_team  
    FROM 
        match m
    GROUP BY 
        m.away_team_api_id
) gt ON gp.away_team_api_id = gt.away_team_api_id, player 
where gp.games_player = gt.total_games_team and player_id = player.player_api_id

player_id,player_name
13390,Wojciech Skaba
105526,Adrian Mrowiec
459662,Volodymyr Tanchyk
555650,Lukasz Moneta
166660,Pieter Nys
641168,Din Sula
138155,Szymon Matuszek
206830,Bartlomiej Pawlowski
412056,Seweryn Michalski
429899,Tomasz Kowalski


In [19]:
%%sql psql

Select count (*)
FROM (
SELECT gp.player_id, player_name --player.player_name
from (
    SELECT away_team_api_id, player_id, count(away_team_api_id) as games_player
    from match, match_player mp
    where mp.team_id = away_team_api_id
    group by away_team_api_id, player_id
    order by away_team_api_id) gp

    JOIN (
    SELECT 
        m.away_team_api_id,
        COUNT(*) AS total_games_team  
    FROM 
        match m
    GROUP BY 
        m.away_team_api_id
) gt ON gp.away_team_api_id = gt.away_team_api_id, player 
where gp.games_player = gt.total_games_team and player_id = player.player_api_id)

count
1288


In [ ]:
%%sql psql

SELECT 
    gp.player_id, 
    player.player_name
FROM (
    SELECT 
        mp.team_id AS away_team_api_id, 
        mp.player_id, 
        COUNT(mp.team_id) AS games_player
    FROM 
        match
    JOIN 
        match_player mp ON mp.team_id = match.away_team_api_id
    GROUP BY 
        mp.team_id, mp.player_id
) gp
JOIN (
    SELECT 
        m.away_team_api_id, 
        COUNT(*) AS total_games_team  
    FROM 
        match m
    GROUP BY 
        m.away_team_api_id
) gt 
ON gp.away_team_api_id = gt.away_team_api_id
JOIN 
    player ON gp.player_id = player.player_api_id
WHERE 
    gp.games_player = gt.total_games_team;


3. What is the match with the highest number of goals? Output `team_long_name` for both home and away teams, and the total number of goals for each team.

In [58]:
%%sql psql
    -- +20 -30 
-- nao estamos a usar o match_player
SELECT 
    M.match_api_id AS match,
    Th.team_long_name AS home_team, 
    M.home_team_goal AS home_team_goals,
    Ta.team_long_name AS away_team, 
    M.away_team_goal AS away_team_goals
FROM Match M 
    JOIN Team Th ON M.home_team_api_id = Th.team_api_id 
    JOIN Team Ta ON M.away_team_api_id = Ta.team_api_id 
WHERE M.home_team_goal + M.away_team_goal >= (
    SELECT MAX(home_team_goal + away_team_goal) 
    FROM match);

match,home_team,home_team_goals,away_team,away_team_goals
820504,Motherwell,6,Hibernian,6
2030233,Real Madrid CF,10,Rayo Vallecano,2


#### 3. Query Optimization

Consider the new `player_attributes` table that contains the largest amount of rows.

1. Create the most appropriate index(es) for the following query: 

In [19]:
%%sql psql
DROP INDEX IF EXISTS player_att_index_c;
DROP INDEX IF EXISTS player_att_idx_hash;
DROP INDEX IF EXISTS player_att_bt;

++
||
++
++

In [11]:
%%sql psql
CREATE INDEX player_att_index_c ON player_attributes(attacking_work_rate, overall_rating)
--8 ms ANSWER makes more sense

++
||
++
++

In [17]:
%%sql psql
CREATE INDEX player_att_idx_hash ON player_attributes USING HASH(attacking_work_rate);
CREATE INDEX player_att_bt ON player_attributes(overall_rating);
-- another option

++
||
++
++

In [18]:
%%sql psql
SELECT
  MAX(overall_rating)
FROM
  player_attributes
WHERE
  attacking_work_rate = 'high';

max
94


In [23]:
%%sql psql
SELECT indexname, tablename, indexdef
FROM pg_indexes
WHERE schemaname = 'public';  -- Pode especificar o schema ou omitir para ver todos os schemas


indexname,tablename,indexdef
client_pkey,client,CREATE UNIQUE INDEX client_pkey ON public.client USING btree (tin)
client_phone_nr_key,client,CREATE UNIQUE INDEX client_phone_nr_key ON public.client USING btree (phone_nr)
assistant_pkey,assistant,CREATE UNIQUE INDEX assistant_pkey ON public.assistant USING btree (phone_nr)
doctor_pkey,doctor,CREATE UNIQUE INDEX doctor_pkey ON public.doctor USING btree (omv)
doctor_phone_nr_key,doctor,CREATE UNIQUE INDEX doctor_phone_nr_key ON public.doctor USING btree (phone_nr)
animal_pkey,animal,"CREATE UNIQUE INDEX animal_pkey ON public.animal USING btree (name, owner)"
cats_or_dogs_pkey,cats_or_dogs,"CREATE UNIQUE INDEX cats_or_dogs_pkey ON public.cats_or_dogs USING btree (name, owner)"
other_animals_pkey,other_animals,"CREATE UNIQUE INDEX other_animals_pkey ON public.other_animals USING btree (name, owner)"
veterinary_visit_pkey,veterinary_visit,"CREATE UNIQUE INDEX veterinary_visit_pkey ON public.veterinary_visit USING btree (name, owner, date)"
assists_pkey,assists,"CREATE UNIQUE INDEX assists_pkey ON public.assists USING btree (name, owner, date, phone_nr)"


2. Create the most appropriate index(es) for the following query: 

In [42]:
%%sql psql
DROP INDEX IF EXISTS idx_id_foot;
DROP INDEX IF EXISTS idx_id_foot_filter;
DROP INDEX IF EXISTS idx_id_foot_filter2;
DROP INDEX IF EXISTS idx_id_foot_filter3;
DROP INDEX IF EXISTS idx_player_attributes_grouping;
DROP INDEX IF EXISTS idx_player_attributes_filtered;


++
||
++
++

In [32]:
%%sql psql
CREATE INDEX idx_id_foot ON player_attributes(player_api_id, preferred_foot);
-- index composto! btree
-- 68 ms

++
||
++
++

In [40]:
%%sql psql
CREATE INDEX idx_id_foot ON player_attributes(player_api_id, preferred_foot);
CREATE INDEX idx_id_foot_a ON player_attributes(crossing, potential);

-- index composto! btree
-- 66 ms ANSWER

++
||
++
++

In [23]:
%%sql psql
DROP INDEX IF EXISTS idx_player_attributes_grouping;
--CREATE INDEX idx_player_attributes_grouping ON player_attributes (crossing, potential, preferred_foot, player_api_id);
-- 84 ms

++
||
++
++

In [24]:
%%sql psql
--EXPLAIN (ANALYZE, BUFFERS) 
EXPLAIN ANALYZE
SELECT
  player_api_id,
  preferred_foot,
  MIN(crossing) as min_crossing, 
  MIN(potential) as min_potential
FROM
  player_attributes
GROUP BY
  player_api_id,
  preferred_foot
HAVING
  MIN(crossing) = 80
  AND MIN(potential) >= 90

-- 77 ms sem having
-- 70 ms com having

QUERY PLAN
HashAggregate (cost=12398.58..12674.48 rows=31 width=29) (actual time=91.356..94.100 rows=2 loops=1)
"Group Key: player_api_id, preferred_foot"
Filter: ((min(crossing) = 80) AND (min(potential) >= 90))
Batches: 1 Memory Usage: 1809kB
Rows Removed by Filter: 12933
-> Seq Scan on player_attributes (cost=0.00..10559.29 rows=183929 width=29) (actual time=0.010..20.781 rows=183929 loops=1)
Planning Time: 0.255 ms
Execution Time: 94.363 ms


## PART III - The Web App (Flask + PostgreSQL)

#### 1. SQL View

1. Create a SQL View that lists the player names sorted by the most recent `player_attributes.date` first.

In [11]:
%%sql psql
--EXPLAIN ANALYZE 
CREATE OR REPLACE VIEW
  players_index_view AS
SELECT pa.player_api_id as player_id, p.player_name, pa.date as date
--SELECT pa.player_fifa_api_id, p.player_name, date
FROM player_attributes pa, player p
WHERE pa.player_api_id = p.player_api_id 
    AND pa.date  = (SELECT MAX(pa2.date) 
    -->= ALL (Select pa2.date
    FROM player_attributes pa2 
    WHERE pa2.player_fifa_api_id = pa.player_fifa_api_id)
-- OR = SELECT MAX(pa2.date) FROM   player_attributes pa2      WHERE pa2.player_fifa_api_id = pa.player_fifa_api_id)
ORDER BY date DESC; --, pa.player_fifa_api_id;
--Finally, regarding the SQL view, we interpret it as listing each player's name only once, corresponding to their most recent date, where the results are sorted by this date. 

++
||
++
++

In [28]:
%%sql psql
--EXPLAIN ANALYZE 
DROP VIEW players_index_view;
CREATE OR REPLACE VIEW
  players_index_view AS
SELECT player_id, p.player_name, date
FROM player p
    JOIN (
SELECT pa.player_api_id as player_id, MAX(pa.date) as date
FROM player_attributes pa
GROUP BY pa.player_api_id
)
ON player_id = p.player_api_id
ORDER BY date DESC;


++
||
++
++

In [105]:
%%sql psql
    --extra query created by me
SELECT * FROM players_index_view
-- 79ms sem indices
-- 81ms jogador, data
-- 75ms data
-- 79 ms data, jogador


-- 212ms sem indices | 162
-- 191 jogador data MORE SENSE
-- 198 date
-- 188 data jogador

player_id,player_name,date
231638,Dzikamai Andre Gwaze,2016-07-07 00:00:00
31432,Joe Hart,2016-06-23 00:00:00
25798,Bruno Lazaroni,2016-06-23 00:00:00
183414,David Stephens,2016-06-16 00:00:00
188058,Daniel Burn,2016-06-16 00:00:00
192625,Patrick Herrmann,2016-06-09 00:00:00
188540,Santiago Vergini,2016-06-09 00:00:00
178584,Sergiusz Prusak,2016-06-09 00:00:00
192319,Jonathan Mensah,2016-06-09 00:00:00
187150,Alejandro Martinuccio,2016-06-09 00:00:00


2. Create the most appropriate index(es) for the SQL view you just created. (Note: PostgreSQL does not have support for materialized views)

In [89]:
%%sql psql
CREATE INDEX idx_player_attributes_date ON player_attributes (date DESC);

++
||
++
++

In [100]:
%%sql psql
CREATE INDEX idx_player_attributes_player_date 
ON player_attributes (player_fifa_api_id, date DESC); 
-- ANSWER

++
||
++
++

In [96]:
%%sql psql
CREATE INDEX idx_player_attributes_player_date 
ON player_attributes (date DESC, player_fifa_api_id); 
-- 12 43

--CREATE INDEX idx_player_api_id ON player(player_api_id); -- este tambem??

++
||
++
++

In [103]:
%%sql psql
DROP INDEX IF EXISTS idx_player_attributes_date;
DROP INDEX IF EXISTS idx_player_api_id;
DROP INDEX IF EXISTS idx_player_attributes_player_date;

++
||
++
++

3. Please test your view using the following query.

In [29]:
%%sql psql
SELECT
  *
FROM
  players_index_view
LIMIT
  20;

player_id,player_name,date
210065,Bebe,2016-07-07 00:00:00
45400,Ronnie Schwartz,2016-07-07 00:00:00
495841,Adalberto Penaranda,2016-07-07 00:00:00
184521,Florian Lejeune,2016-07-07 00:00:00
423024,Jhon Murillo,2016-07-07 00:00:00
317932,Baptiste Aloe,2016-07-07 00:00:00
307224,Kevin Koubemba,2016-07-07 00:00:00
512726,Yanis Mbombo Lokwa,2016-07-07 00:00:00
450002,Wallace,2016-07-07 00:00:00
167254,Jo-Gook Jung,2016-06-30 00:00:00


#### 2. Create the players_index page on the Web App

Requirements:
- Should use the SQL View declared above
- Do not list more than 20 players.

1. Copy your players_index function from app.py to the cell below

In [ ]:
# rever o que está aqui depois desta célula

```python
@app.route("/", methods=("GET",))
@app.route("/players", methods=("GET",))
@limiter.limit("1 per second")
def players_index():
    """Show the list of players sorted by most recent player attributes date first."""
    
    with pool.connection() as conn:
        with conn.cursor() as cur:
            # Querying the SQL View to get player information
            players = cur.execute(
                """
                SELECT player_api_id, player_name, date
                FROM player_attributes_view
                ORDER BY date DESC
                LIMIT 20;
                """,
                {}
            ).fetchall()
            log.debug(f"Found {cur.rowcount} players.")

    return render_template("players_index.html", players=players)
```

```python
@app.route("/", methods=("GET",))
@app.route("/accounts", methods=("GET",))
@limiter.limit("1 per second")
def account_index():
    """Show all the accounts, most recent first."""

    with pool.connection() as conn:
        with conn.cursor() as cur:
            accounts = cur.execute(
                """
                SELECT account_number, branch_name, balance
                FROM account
                ORDER BY account_number DESC;
                """,
                {},
            ).fetchall()
            log.debug(f"Found {cur.rowcount} rows.")

    return render_template("account/index.html", accounts=accounts)
```

2. Copy your final HTML template for this view onto the cell below

```html
{% extends "base.html" %} <!-- confirm name.html-->
{% block header %}
    <h1>
        {% block title %}
            Players
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}

    {% for player in players %}
        <article class="post">
            <header>
                <div>
                    <h1>{{ player['player_name'] }}</h1>
                    <div class="about">Last Updated: {{ player['date'] }}</div>  <!-- Data da última atualização -->
                </div>
                <a class="action"
                    href="{{ url_for('players_index_view', player_api_id=player['player_api_id']) }}">View Details</a>
            </header>
            <!--<p class="body">€ {{ account['balance'] }}</p> -->
        </article>
        {% if not loop.last %}<hr />{% endif %}
    {% endfor %}
```

```html
{% extends "base.html" %} <!-- confirm name.html-->
{% block header %}
    <h1>
        {% block title %}
            Players
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}

    <div class="players-list">
        {% if players %}
            <ul>
                {% for player in players %}
                    <li>
                        <article class="post">
                            <header>
                                <div>
                                    <h1>{{ player['player_name'] }}</h1>  
                                    <div class="about">Last Updated: {{ player['date'] }}</div>  
                                </div>
                                <a class="action" href="{{ url_for('players_index_view', player_api_id=player['player_fifa_api_id']) }}">View Details</a>
                            </header>
                        </article>
                    </li>
                    {% if not loop.last %}<hr />{% endif %}
                {% endfor %}
            </ul>
        {% else %}
            <p>No players found.</p>
        {% endif %}
    </div>
{% endblock content %}
                                       
```

```html
{% extends "base.html" %}
{% block header %}
    <h1>
        {% block title %}
            Accounts
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}
    {% for account in accounts %}
        <article class="post">
            <header>
                <div>
                    <h1>{{ account['account_number'] }}</h1>
                    <div class="about">in {{ account['branch_name'] }}</div>
                </div>
                <a class="action"
                   href="{{ url_for('account_update_view', account_number=account['account_number']) }}">Edit</a>
            </header>
            <p class="body">€ {{ account['balance'] }}</p>
        </article>
        {% if not loop.last %}<hr />{% endif %}
    {% endfor %}
{% endblock content %}
```

#### 3. Create the player_update_view and player_update_save routes on the Web App

Requirements:

- The app.route for player_update_view should take a player_api_id and show a nice editable HTML form.
             
- The app.route for player_update_save should be called on form submission and insert the new data into player_attributes. 

- On success, should redirect to the players_index

- On failure, should raise an appropriate exception with a error message

1. Copy your player_update_view function from app.py to the cell below

In [107]:
%%sql psql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'player_attributes';

column_name,data_type
gk_reflexes,bigint
player_fifa_api_id,bigint
player_api_id,bigint
gk_handling,bigint
gk_kicking,bigint
gk_positioning,bigint
id,bigint
overall_rating,bigint
potential,bigint
crossing,bigint


In [108]:
%%sql psql
SELECT count(column_name)
FROM information_schema.columns
WHERE table_name = 'player_attributes';

count
42


2. Copy your final HTML template for this view onto the cell below

3. Copy your player_update_save function from app.py to the cell below